# SymPy demo - prosta symulacja

* Krzysztof Molenda, v0.1, 2025-11-16

* Tutorial SimPy: <https://simpy.readthedocs.io/en/latest/simpy_intro/index.html>

Stworzymy symulację, w której klienci przychodzą do sklepu, robią zakupy przez określony czas, następnie podchodzą do kasy, są obsługiwani i opuszczają sklep. Każdy etap zostanie zamodelowany jako proces w środowisku SimPy z użyciem klasy.

In [ ]:
import simpy

## Krok 1 - Definiowanie klasy `Customer`

Tworzymy klasę Customer, która będzie reprezentować pojedynczego klienta i przebieg jego wizyty w sklepie.

In [ ]:
class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time):
        self.env = env                    # środowisko symulacyjne
        self.name = name                  # nazwa/identyfikator klienta
        self.shopping_time = shopping_time # czas spędzony na zakupach
        self.checkout_time = checkout_time # czas obsługi przy kasie
        self.action = env.process(self.run()) # uruchomienie procesu symulacyjnego

Wyjaśnienie:
- Konstruktor klasy przyjmuje środowisko symulacyjne (`env`), nazwę klienta (`name`) oraz dwa parametry czasowe dotyczące etapów wizyty (`shopping_time`, `checkout_time`).
- Proces symulacyjny uruchamiany jest poprzez `env.process(self.run())`. Dzięki temu każdy klient „wykonuje” własny scenariusz działania.

## Krok 2 - Modelowanie przebiegu wizyty klienta – metoda `run()`

Do klasy `Customer` dodajemy metodę `run()` - zapowiedzianą w konstruktorze, która opisuje sekwencję zdarzeń związanych z wizytą klienta:

:::{tip} Kod
```python
class Customer(object):
    ...

    def run(self):
        print(f"{self.name} przychodzi do sklepu w czasie {self.env.now}")
        yield self.env.timeout(self.shopping_time)
        
        print(f"{self.name} podchodzi do kasy w czasie {self.env.now}")
        yield self.env.timeout(self.checkout_time)
        
        print(f"{self.name} opuszcza sklep w czasie {self.env.now}")
```
:::

- Klient rozpoczyna wizytę: wyświetlana jest informacja o czasie przyjścia (`env.now`).
- Następnie klient „spędza” w sklepie czas zakupów (`yield env.timeout(self.shopping_time)`).
- Kolejny etap to podejście do kasy, obsługa przez określony czas i opuszczenie sklepu.
- `yield env.timeout(x)` powoduje upływ czasu symulacyjnego "na żądanie".

:::{note} Środowisko env
:class: dropdown

Środowisko `simpy.Environment()` to centralny komponent każdej symulacji zdarzeń dyskretnych. Odpowiada za zarządzanie czasem symulacji oraz koordynacją przebiegu wszystkich procesów i zdarzeń. Jest to obiekt,  który symuluje upływ czasu i reguluje przebieg wszystkich operacji w modelu ("silnik dla symulacji"):

- uruchamia procesy (np. klientów, maszyny, pojazdy),
- przechowuje wszystkie zdarzenia,
- dba o to, aby procesy wykonywane były w odpowiedniej kolejności i we właściwym symulowanym czasie.

Zadania środowiska symulacyjnego w SimPy
1. Zarządzanie czasem symulacji
    - Środowisko przechowuje aktualny czas symulacji: `env.now`.
    - Każde zdarzenie (np. `timeout`, żądanie zasobu, zwolnienie zasobu) odbywa się w określonym czasie symulacyjnym.
    - Kiedy uruchamiasz `env.run(until=X)`, środowisko „przesuwa się w czasie” od `0` do `X`, wywołując zdarzenia chronologicznie.

2. Planowanie i obsługa procesów
    - Procesy (czyli funkcje lub metody, które modelują zachowania) są uruchamiane w środowisku za pomocą `env.process(...)`.
    - Środowisko pilnuje, by każdy proces został „wybudzony” do działania w odpowiednim momencie i na właściwą ilość czasu (np. czekanie na zakończenie zakupów, obsługę przy kasie).

3. Synchronizowanie działań i interakcji
    - Dzięki środowisku możliwe jest tworzenie modeli, gdzie wiele „aktorów” współistnieje, wchodzi ze sobą w interakcje (np. ustawianie się w kolejce, korzystanie z tego samego zasobu/kasy), a wszystkie zdarzenia są zsynchronizowane w kontekście wspólnego czasu symulacji.
:::

::::{note} yield
:class: dropdown

W kontekście SimPy słowo kluczowe `yield` oznacza przekazanie kontroli nad przebiegiem procesu środowisku symulacyjnemu. Jego użycie pozwala na „zawieszenie” procesu w określonym punkcie, aż wystąpi odpowiednie zdarzenie lub upłynie określony czas.

- W SimPy procesy są _Pythonowymi generatorami_ (Pythonowe funkcje lub metody z `yield`), które mogą „pauzować” i „wznawiać” swoje wykonanie zależnie od zdarzeń zachodzących w symulacji.
- Gdy w procesie pojawia np. `yield env.timeout(5)`, proces:
    - zawiesza swoje działanie na 5 jednostek czasu symulacji,
    - w tym czasie mogą pracować inne procesy lub mogą zachodzić inne zdarzenia,
    - po upływie 5 jednostek procesu zostaje wznowiony i kontynuuje działanie od miejsca, gdzie został „zatrzymany”.

:::{tip} Przykład
```python
def client(env):
    print(f"Klient rozpoczyna zakupy w czasie {env.now}")
    yield env.timeout(4)    # „pauza” na 4 jednostki czasu
    print(f"Klient podchodzi do kasy w czasie {env.now}")
```

Najpierw klient robi zakupy, potem czeka `4` jednostki czasu symulacji (tu sterowanie przejmuje środowisko `env`), a następnie wykonuje kolejną instrukcję po „wybudzeniu”.
:::
::::

In [ ]:
# kod klasy Customer uzupełniony o metodę run()
class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time):
        self.env = env                        # środowisko symulacyjne
        self.name = name                      # nazwa/identyfikator klienta
        self.shopping_time = shopping_time    # czas spędzony na zakupach
        self.checkout_time = checkout_time    # czas obsługi przy kasie
        self.action = env.process(self.run()) # uruchomienie procesu symulacyjnego
        
    def run(self):
        print(f"{self.name} przychodzi do sklepu w czasie {self.env.now}")
        yield self.env.timeout(self.shopping_time)
        
        print(f"{self.name} podchodzi do kasy w czasie {self.env.now}")
        yield self.env.timeout(self.checkout_time)
        
        print(f"{self.name} opuszcza sklep w czasie {self.env.now}")

## Krok 3 - Utworzenie środowiska symulacyjnego

Środowisko jest „silnikiem” czasu i procesów symulacji.

```python
env = simpy.Environment()
```

Tworzymy kilku klientów, którzy wejdą do sklepu i przejdą przez zaplanowane etapy.

```python
c1 = Customer(env, "Klient 1", shopping_time=4, checkout_time=2)
c2 = Customer(env, "Klient 2", shopping_time=3, checkout_time=1)
c3 = Customer(env, "Klient 3", shopping_time=5, checkout_time=3)
```

- Każdy klient może mieć inne czasy spędzone na różnych etapach wizyty.
- Parametry przekazane do obiektów klasy pozwalają modelować różnorodność zachowań.

In [ ]:
env = simpy.Environment()

c1 = Customer(env, "Klient 1", shopping_time=4, checkout_time=2)
c2 = Customer(env, "Klient 2", shopping_time=3, checkout_time=1)
c3 = Customer(env, "Klient 3", shopping_time=5, checkout_time=3)

## Krok 4 - uruchomienie symulacji

Nastepnie uruchamiamy środowisko, podając czas końca (np. 15 jednostek czasu).

In [ ]:
env.run(until=15)

## Krok 4 - rozbudowa, przerwanie procesu

Proces zakupów przez klienta może byc przerwany z różnych powodów (np. rezygnacja z zakupów, wezwanie na parkin, alarm przeciwpożarowy, interwencja ochrony, ...).

Formalnie - jeden proces może być przerwany przez inny proces.

W SimPy stosujemy mechanizm `env.process(...).interrupt()`, który pozwala „przerwać” aktualne oczekiwanie (`yield env.timeout(...)`) w procesie klienta. Obsługa przerwania realizowana jest w bloku `try/except simpy.Interrupt`, gdzie można zaprogramować, co klient robi po przerwaniu (np. opuszcza sklep, idzie do kasy szybciej, reaguje na alarm).

Zmiana kodu klasy `Customer` polega na:

- modyfikacji metody `run` dodając obsługę wyjątku `simpy.Interrup`
- dodaniu w skrypcie symulacyjnym funkcji `interrupter()` definiującej przerwanie

In [ ]:
import simpy

class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time):
        self.env = env
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.action = env.process(self.run())

    def run(self):
        try:
            print(f"{self.name} przychodzi do sklepu w czasie {self.env.now}")
            yield self.env.timeout(self.shopping_time)  # zakupy
            print(f"{self.name} podchodzi do kasy w czasie {self.env.now}")
            yield self.env.timeout(self.checkout_time)  # obsługa przy kasie
            print(f"{self.name} opuszcza sklep w czasie {self.env.now}")
        except simpy.Interrupt as interrupt:
            print(f"{self.name}: zakupy przerwane w czasie {self.env.now} ({interrupt.cause})")
            # Tu klient może np. od razu udać się do kasy lub opuścić sklep
            print(f"{self.name} natychmiast opuszcza sklep w czasie {self.env.now}")

In [ ]:
def interrupter(env, customer, interrupt_at, reason):
    yield env.timeout(interrupt_at)
    customer.action.interrupt(cause=reason)

Przeprowadźmy eksperyment z przerwaniem zakupów przez klienta nr 1:

In [ ]:
env = simpy.Environment()

c1 = Customer(env, "Klient 1", shopping_time=10, checkout_time=2)

# Dodajemy proces, który przerwie klientowi zakupy po 5 jednostkach czasu
env.process(interrupter(env, c1, interrupt_at=5, reason="Wezwanie na parking z powodu stłuczki"))

env.run(until=15)

- Proces klienta podczas zakupów może zostać przerwany przez wyjątek `simpy.Interrupt` (np. wezwanie na parking).
- Proces `interrupter` wywołuje przerwanie dla klienta po zadanym czasie i z podanym powodem.
- Po przerwaniu można zaprogramować dowolną reakcję klienta.

## Krok 5 - rozbudowa, różne reakcje klientów na przerwanie

Rozważmy nastepującą sytuację: Każdy klient po przerwaniu może podjąć inną decyzję (np. natychmiast opuszcza sklep, idzie do kasy mimo przerwania, ignoruje przerwanie, ...).

Musimy rozbudować metodę `run()` w klasie `Customer`, uwzględniając różne przyczyny przerwania. W konstruktorze klasy `Customer` dodajemy atrybut `reaction` określający zachowanie się klienta po przerwaniu.

Opis predefiniowanych zachowań po przerwaniu:
- "leave" — klient natychmiast opuszcza sklep
- "checkout" — klient udaje się bezpośrednio do kasy i jest obsłużony
- "ignore" — klient ignoruje przerwanie i kontynuuje zakupy, jakby nic się nie stało

In [ ]:
class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time, reaction):
        self.env = env
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.reaction = reaction  # określa zachowanie po przerwaniu
        self.action = env.process(self.run())

    def run(self):
        try:
            print(f"{self.name} przychodzi do sklepu w czasie {self.env.now}")
            yield self.env.timeout(self.shopping_time)  # zakupy
            print(f"{self.name} podchodzi do kasy w czasie {self.env.now}")
            yield self.env.timeout(self.checkout_time)  # obsługa przy kasie
            print(f"{self.name} opuszcza sklep w czasie {self.env.now}")
        except simpy.Interrupt as interrupt:
            print(f"{self.name} - zakupy przerwane ({interrupt.cause}) w czasie {self.env.now}")
            # Różne reakcje na przerwanie
            if self.reaction == "leave":
                print(f"{self.name} natychmiast opuszcza sklep w czasie {self.env.now}")
            elif self.reaction == "checkout":
                print(f"{self.name} od razu podchodzi do kasy mimo przerwania w czasie {self.env.now}")
                yield self.env.timeout(self.checkout_time)
                print(f"{self.name} opuszcza sklep (po kasie) w czasie {self.env.now}")
            elif self.reaction == "ignore":
                print(f"{self.name} ignoruje przerwanie i kontynuuje zakupy w czasie {self.env.now}")
                # Kontynuuje poprzedni czas: dla uproszczenia idzie do kasy jak planował
                yield self.env.timeout(max(0, self.shopping_time - (self.env.now)))  # resztę czasu
                print(f"{self.name} podchodzi do kasy w czasie {self.env.now}")
                yield self.env.timeout(self.checkout_time)
                print(f"{self.name} opuszcza sklep w czasie {self.env.now}")

Tworzymy kilku klientów z różnymi reakcjami na przerwanie (klient 4 zachowuje się losowo), umieszczamy ich w liście.

Przerywamy zakupy klientom po różnych czasach

In [ ]:
import random
env = simpy.Environment()

# Tworzymy kilku klientów z różnymi reakcjami na przerwanie
customers = [
    Customer(env, "Klient 1", shopping_time=6, checkout_time=2, reaction="leave"),
    Customer(env, "Klient 2", shopping_time=7, checkout_time=3, reaction="checkout"),
    Customer(env, "Klient 3", shopping_time=5, checkout_time=1, reaction="ignore"),
    Customer(env, "Klient 4", shopping_time=10, checkout_time=2, reaction=random.choice(["leave", "checkout", "ignore"])),  # losowo
]

# Przerywamy zakupy klientom po różnych czasach
env.process(interrupter(env, customers[0], interrupt_at=3, reason="Alarm przeciwpożarowy"))
env.process(interrupter(env, customers[1], interrupt_at=4, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=2, reason="Awaria techniczna"))
env.process(interrupter(env, customers[3], interrupt_at=6, reason="Ogłoszenie specjalne"))

env.run(until=20)

## Krok 6 - współdzielone zasoby

Współdzielone zasoby (_shared resources_) są kluczowym elementem SymPy, pozwalającym modelować np. sytuację, gdy kilku klientów korzysta z tej samej kasy lub kilku kas, a ich liczba jest ograniczona. 
Typowe zasoby w SimPy to: `Resource` (ogólne stanowiska), `Store` (magazyny), `Container` (pojemności), `PriorityResource` (zasoby z priorytetami) itd.

Rozważmy sytuację, gdy klienci czekają w kolejce ograniczonej liczby kas. Kasa jest współdzielonym zasobem.

Modyfikujemy klsę `Customer`:

1. Konstruktor klasy przyjmuje dodatkowy argument `checkout_resource`, czyli wskaźnik na współdzieloną kasę (np. obiekt `Resource`). Do tej kasy klient podchodzi i ustawia się w kolejce
   ```python
   def __init__(self, env, name, shopping_time, checkout_time, reaction, checkout_resource):
    ...
    self.checkout_resource = checkout_resource
   ```
2. W metodzie `run()` dodajemy się blok:
   ```python
   with self.checkout_resource.request() as req:
       yield req
       # teraz obsługa przy kasie
       yield self.env.timeout(self.checkout_time)
   ```
   Klient „żąda” dostępu do zasobu (kasy). Jeśli wszystkie stanowiska są zajęte, czeka w kolejce, aż kasa się zwolni. Dopiero wtedy następuje obsługa.
3. W sytuacjach przerwania, musimy również opisać obsługę żądania zasobu - różne reakcje na przerwanie (np. klient idzie do kasy po przerwaniu lub ignoruje przerwanie i kończy zakupy)
   ```python
   with self.checkout_resource.request() as req:
       yield req
       # obsługa przy kasie jak wyżej
   ```
   Blok `with self.checkout_resource.request() as req` – klient żąda dostępu do zasobu (kasy), jeśli nie ma wolnego miejsca, czeka w kolejce. 

   Umożliwia to poprawne modelowanie dostępu do wspólnej kasy nawet w nietypowych ścieżkach wykonywania procesu (np. po przerwaniu zakupów klient od razu idzie do kasy lub kontynuuje normalnie, ale zawsze korzysta z tego samego zasobu).

In [ ]:
class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time, reaction, checkout_resource):
        self.env = env
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.reaction = reaction
        self.checkout_resource = checkout_resource  # wskazany zasób (kasa)
        self.action = env.process(self.run())

    def run(self):
        try:
            print(f"{self.name} przychodzi do sklepu w czasie {self.env.now}")
            yield self.env.timeout(self.shopping_time)

            print(f"{self.name} podchodzi do kasy w czasie {self.env.now} | czeka na wolną kasę")
            with self.checkout_resource.request() as req:
                yield req  # Czekaj w kolejce do zasobu (kasy)
                print(f"{self.name} rozpoczyna obsługę przy kasie w czasie {self.env.now}")
                yield self.env.timeout(self.checkout_time)
                print(f"{self.name} opuszcza sklep w czasie {self.env.now}")

        except simpy.Interrupt as interrupt:
            print(f"{self.name} - zakupy przerwane ({interrupt.cause}) w czasie {self.env.now}")
            if self.reaction == "leave":
                print(f"{self.name} natychmiast opuszcza sklep w czasie {self.env.now}")
            elif self.reaction == "checkout":
                # Klient jednak idzie do kasy po przerwaniu
                print(f"{self.name} od razu podchodzi do kasy (przerwanie) w czasie {self.env.now}")
                with self.checkout_resource.request() as req:
                    yield req
                    print(f"{self.name} (przerwany) obsługiwany przy kasie w czasie {self.env.now}")
                    yield self.env.timeout(self.checkout_time)
                    print(f"{self.name} opuszcza sklep (po kasie) w czasie {self.env.now}")
            elif self.reaction == "ignore":
                print(f"{self.name} ignoruje przerwanie i kontynuuje zakupy...")
                yield self.env.timeout(max(0, self.shopping_time - (self.env.now)))
                print(f"{self.name} (po przerwaniu) podchodzi do kasy w czasie {self.env.now}")
                with self.checkout_resource.request() as req:
                    yield req
                    print(f"{self.name} (po przerwaniu) obsługiwany przy kasie w czasie {self.env.now}")
                    yield self.env.timeout(self.checkout_time)
                    print(f"{self.name} opuszcza sklep w czasie {self.env.now}")

Tworząc kod symulacji musimy stworzyć współdzielone zasoby (w naszym rzypadku typu `Resource`:

```python
cashiers = simpy.Resource(env, capacity=1)  # 1 stanowisko kasowe
```

i tworząc użytkowników w konstruktorze wskazać współdzielony zasób

```python
customers = [
    Customer(env, ..., checkout_resource=cashiers),
    ...
]
```

In [ ]:
# symulacja
env = simpy.Environment()
cashiers = simpy.Resource(env, capacity=1)  # 1 stanowisko kasowe

customers = [
    Customer(env, "Klient 1", shopping_time=6, checkout_time=2, reaction="leave", checkout_resource=cashiers),
    Customer(env, "Klient 2", shopping_time=7, checkout_time=3, reaction="checkout", checkout_resource=cashiers),
    Customer(env, "Klient 3", shopping_time=5, checkout_time=1, reaction="ignore", checkout_resource=cashiers),
    Customer(env, "Klient 4", shopping_time=10, checkout_time=3, reaction=random.choice(["leave", "checkout", "ignore"]), checkout_resource=cashiers),
]

# Przerywamy zakupy klientom w różnych momentach
env.process(interrupter(env, customers[0], interrupt_at=3, reason="Alarm przeciwpożarowy"))
env.process(interrupter(env, customers[1], interrupt_at=4, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=2, reason="Awaria techniczna"))
env.process(interrupter(env, customers[3], interrupt_at=6, reason="Ogłoszenie specjalne"))

env.run(until=30)

Możemy zwiększyć liczbę kas:

```python
cashiers = simpy.Resource(env, capacity=2)  # 2 stanowisko kasowe
```
i powtórzyć symulację. Klienci oczekują w jednej kolejce, gdy oba zasoby są zajęte.

In [ ]:
# symulacja
env = simpy.Environment()
cashiers = simpy.Resource(env, capacity=2)  # 2 stanowiska kasowe

customers = [
    Customer(env, "Klient 1", shopping_time=6, checkout_time=2, reaction="leave", checkout_resource=cashiers),
    Customer(env, "Klient 2", shopping_time=7, checkout_time=3, reaction="checkout", checkout_resource=cashiers),
    Customer(env, "Klient 3", shopping_time=5, checkout_time=1, reaction="ignore", checkout_resource=cashiers),
    Customer(env, "Klient 4", shopping_time=10, checkout_time=3, reaction=random.choice(["leave", "checkout", "ignore"]), checkout_resource=cashiers),
]

# Przerywamy zakupy klientom w różnych momentach
env.process(interrupter(env, customers[0], interrupt_at=3, reason="Alarm przeciwpożarowy"))
env.process(interrupter(env, customers[1], interrupt_at=4, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=2, reason="Awaria techniczna"))
env.process(interrupter(env, customers[3], interrupt_at=6, reason="Ogłoszenie specjalne"))

env.run(until=30)

## Krok 7 - rejestrowanie danych symulacji

We wcześniejszym przykładzie przebieg symulacji raportowany był wydrukami. W praktyce przebieg ten rejestrujemy np. w plikach, które później poddawane są obróbce.

### Zbieranie danych do struktur zamiast `print`

Zamiast wypisywać komunikaty bezpośrednio, można dodawać je do listy, słownika lub dataframe (np. pandas). Po zakończeniu symulacji będzie można przeanalizować i wyświetlać wyniki w uporządkowanej formie.

```python
class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time, reaction, checkout_resource):
        ...
        self.log = []

    def run(self):
        try:
            self.log.append((self.env.now, f"{self.name} przychodzi do sklepu"))
            yield self.env.timeout(self.shopping_time)

            self.log.append((self.env.now, f"{self.name} podchodzi do kasy"))
            with self.checkout_resource.request() as req:
                yield req
                self.log.append((self.env.now, f"{self.name} rozpoczyna obsługę przy kasie"))
                yield self.env.timeout(self.checkout_time)
                self.log.append((self.env.now, f"{self.name} opuszcza sklep"))

        except simpy.Interrupt as interrupt:
            self.log.append((self.env.now, f"{self.name} przerwany ({interrupt.cause})"))
            # pozostałe wpisy zgodnie z reakcją...

...
# symulacja
results = [] # dopisujemy zdarzenia do listy
             # elementami listy są "czas | zdarzenie"
             # separator: "|"
# Po zakończeniu symulacji:
for c in customers:
    for event in c.log:
        results.append(event)
# Posortowanie i wypisanie chronologiczne
for t, desc in sorted(results):
    print(f"{t:>4} | {desc}")
```

In [ ]:
class Customer(object):
    def __init__(self, env, name, shopping_time, checkout_time, reaction, checkout_resource):
        self.env = env
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.reaction = reaction
        self.checkout_resource = checkout_resource  # wskazany zasób (kasa)
        self.action = env.process(self.run())
        self.log = []

    def run(self):
        try:
            self.log.append((self.env.now, f"{self.name} przychodzi do sklepu"))
            yield self.env.timeout(self.shopping_time)

            self.log.append((self.env.now, f"{self.name} podchodzi do kasy"))
            with self.checkout_resource.request() as req:
                yield req
                self.log.append((self.env.now, f"{self.name} rozpoczyna obsługę przy kasie"))
                yield self.env.timeout(self.checkout_time)
                self.log.append((self.env.now, f"{self.name} opuszcza sklep"))

        except simpy.Interrupt as interrupt:
            self.log.append((self.env.now, f"{self.name} przerwany ({interrupt.cause})"))
            # pozostałe wpisy zgodnie z reakcją...
            print(f"{self.name} - zakupy przerwane ({interrupt.cause}) w czasie {self.env.now}")
            if self.reaction == "leave":
                print(f"{self.name} natychmiast opuszcza sklep w czasie {self.env.now}")
            elif self.reaction == "checkout":
                # Klient jednak idzie do kasy po przerwaniu
                print(f"{self.name} od razu podchodzi do kasy (przerwanie) w czasie {self.env.now}")
                with self.checkout_resource.request() as req:
                    yield req
                    print(f"{self.name} (przerwany) obsługiwany przy kasie w czasie {self.env.now}")
                    yield self.env.timeout(self.checkout_time)
                    print(f"{self.name} opuszcza sklep (po kasie) w czasie {self.env.now}")
            elif self.reaction == "ignore":
                print(f"{self.name} ignoruje przerwanie i kontynuuje zakupy...")
                yield self.env.timeout(max(0, self.shopping_time - (self.env.now)))
                print(f"{self.name} (po przerwaniu) podchodzi do kasy w czasie {self.env.now}")
                with self.checkout_resource.request() as req:
                    yield req
                    print(f"{self.name} (po przerwaniu) obsługiwany przy kasie w czasie {self.env.now}")
                    yield self.env.timeout(self.checkout_time)
                    print(f"{self.name} opuszcza sklep w czasie {self.env.now}")

In [ ]:
# symulacja
env = simpy.Environment()
cashiers = simpy.Resource(env, capacity=2)  # 2 stanowiska kasowe

results = []
customers = [
    Customer(env, "Klient 1", shopping_time=6, checkout_time=2, reaction="leave", checkout_resource=cashiers),
    Customer(env, "Klient 2", shopping_time=7, checkout_time=3, reaction="checkout", checkout_resource=cashiers),
    Customer(env, "Klient 3", shopping_time=5, checkout_time=1, reaction="ignore", checkout_resource=cashiers),
    Customer(env, "Klient 4", shopping_time=10, checkout_time=3, reaction=random.choice(["leave", "checkout", "ignore"]), checkout_resource=cashiers),
]

# Przerywamy zakupy klientom w różnych momentach
env.process(interrupter(env, customers[0], interrupt_at=3, reason="Alarm przeciwpożarowy"))
env.process(interrupter(env, customers[1], interrupt_at=4, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=2, reason="Awaria techniczna"))
env.process(interrupter(env, customers[3], interrupt_at=6, reason="Ogłoszenie specjalne"))

env.run(until=30)

# Po zakończeniu symulacji:
print("WYNIKI:")
for c in customers:
    for event in c.log:
        results.append(event)
# Posortowanie i wypisanie chronologiczne
for t, desc in sorted(results):
    print(f"{t:>4} | {desc}")

### Załadowanie listy zdarzeń do `pandas`

Wygodniejszym sposobem przetwarzania danych w liście logów będzie posłużenie się biblioteką `pandas`:

In [ ]:
import pandas as pd

df = pd.DataFrame(results, columns=["czas", "zdarzenie"])
print(df.sort_values("czas").to_string(index=False))

### Grupowanie logów - każdy klient w oddzielnej sekcji

Możemy wypisać logi dla każdego klienta oddzielnie.

In [ ]:
for c in customers:
    print(f"\nLOG {c.name}:")
    for t, desc in c.log:
        print(f"  {t:>4} | {desc}")

### Rozszerzone informacje w logach

W przypadku potrzeby zbierania w logach innych informacji, np. czas oczekiwania w kolejce, czas zakupów, cza obsługi), konieczne będzie zmodyfikowanie klasy `Customer` o stosowne parametry, oraz ponowne zdefiniowanie logiki w metodzie `run()`.

In [ ]:
class Customer(object):
    def __init__(self, env, customer_id, name, shopping_time, checkout_time, reaction, checkout_resource):
        self.env = env
        self.customer_id = customer_id
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.reaction = reaction
        self.checkout_resource = checkout_resource
        
        self.log = []
        self.summary = {}
        self.action = env.process(self.run())

    def run(self):
        arrival_time = self.env.now
        try:
            self.log.append((self.env.now, f"[{self.customer_id}] {self.name} przychodzi do sklepu"))
            yield self.env.timeout(self.shopping_time)
            shopping_end = self.env.now
            self.log.append((self.env.now, f"[{self.customer_id}] {self.name} kończy zakupy, podchodzi do kasy"))

            with self.checkout_resource.request() as req:
                queue_start = self.env.now
                yield req
                queue_time = self.env.now - queue_start
                cashier_number = self.checkout_resource.count  # ile kas zajętych
                self.log.append((self.env.now, f"[{self.customer_id}] {self.name} zaczyna obsługę przy kasie {cashier_number}"))
                yield self.env.timeout(self.checkout_time)
                self.log.append((self.env.now, f"[{self.customer_id}] {self.name} opuszcza sklep"))

                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": shopping_end - arrival_time,
                    "czas_kolejki": queue_time,
                    "czas_obsługi": self.checkout_time,
                    "czas_w_sklepie": self.env.now - arrival_time,
                    "status": "ukończone"
                }
        except simpy.Interrupt as interrupt:
            interrupt_time = self.env.now
            self.log.append((self.env.now, f"[{self.customer_id}] {self.name} PRZERWANIE ({interrupt.cause})"))
            czas_zakupow = interrupt_time - arrival_time
            if self.reaction == "leave":
                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": czas_zakupow,
                    "czas_kolejki": "-",
                    "czas_obsługi": "-",
                    "czas_w_sklepie": czas_zakupow,
                    "status": "przerwane i opuszczone"
                }
            elif self.reaction == "checkout":
                start_kasy = self.env.now
                self.log.append((self.env.now, f"[{self.customer_id}] {self.name} od razu podchodzi do kasy (przerwanie)"))
                with self.checkout_resource.request() as req:
                    yield req
                    queue_time = self.env.now - start_kasy
                    cashier_number = self.checkout_resource.count
                    self.log.append((self.env.now, f"[{self.customer_id}] {self.name} (przerwany) obsługiwany przy kasie {cashier_number}"))
                    yield self.env.timeout(self.checkout_time)
                    koniec = self.env.now
                    self.log.append((self.env.now, f"[{self.customer_id}] {self.name} opuszcza sklep (po kasie)"))
                    self.summary = {
                        "id": self.customer_id,
                        "nazwa": self.name,
                        "czas_zakupow": czas_zakupow,
                        "czas_kolejki": queue_time,
                        "czas_obsługi": self.checkout_time,
                        "czas_w_sklepie": koniec - arrival_time,
                        "status": "przerwane, potem obsługa"
                    }
            elif self.reaction == "ignore":
                remain_shopping = max(0, self.shopping_time - czas_zakupow)
                self.log.append((self.env.now, f"[{self.customer_id}] {self.name} ignoruje przerwanie, kontynuuje zakupy..."))
                yield self.env.timeout(remain_shopping)
                koniec_zakupow = self.env.now
                self.log.append((self.env.now, f"[{self.customer_id}] {self.name} (po przerwaniu) podchodzi do kasy"))
                with self.checkout_resource.request() as req:
                    queue_start = self.env.now
                    yield req
                    queue_time = self.env.now - queue_start
                    cashier_number = self.checkout_resource.count
                    self.log.append((self.env.now, f"[{self.customer_id}] {self.name} (po przerwaniu) obsługiwany przy kasie {cashier_number}"))
                    yield self.env.timeout(self.checkout_time)
                    koniec = self.env.now
                    self.log.append((self.env.now, f"[{self.customer_id}] {self.name} opuszcza sklep (po kasie)"))
                    self.summary = {
                        "id": self.customer_id,
                        "nazwa": self.name,
                        "czas_zakupow": self.shopping_time,  # planowany czas zakupów
                        "czas_kolejki": queue_time,
                        "czas_obsługi": self.checkout_time,
                        "czas_w_sklepie": koniec - arrival_time,
                        "status": "przerwane, ignorowane"
                    }

In [ ]:
# symulacja
env = simpy.Environment()
cashiers = simpy.Resource(env, capacity=2)

customers = [
    Customer(env, 1, "Anna", shopping_time=6, checkout_time=2, reaction="leave", checkout_resource=cashiers),
    Customer(env, 2, "Bartek", shopping_time=7, checkout_time=3, reaction="checkout", checkout_resource=cashiers),
    Customer(env, 3, "Celina", shopping_time=5, checkout_time=1, reaction="ignore", checkout_resource=cashiers),
    Customer(env, 4, "Darek", shopping_time=10, checkout_time=3,
             reaction=random.choice(["leave", "checkout", "ignore"]), checkout_resource=cashiers),
]
env.process(interrupter(env, customers[0], interrupt_at=3, reason="Alarm przeciwpożarowy"))
env.process(interrupter(env, customers[1], interrupt_at=4, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=2, reason="Awaria techniczna"))
env.process(interrupter(env, customers[3], interrupt_at=6, reason="Ogłoszenie specjalne"))

env.run(until=30)

# Czytelne logi
print("="*40)
print("LOGI SYMULACJI")
print("="*40)
for c in customers:
    print(f"\n[Klient {c.customer_id}] {c.name}:")
    for t, desc in c.log:
        print(f"  {t:>4} | {desc}")

print("\n" + "="*65)
print("PODSUMOWANIE KLIENTÓW")
print("="*65)
print(f"{'ID':>2} {'Nazwa':<10} {'Zakupy':>8} {'Kolejka':>8} {'Obsługa':>8} {'W sklepie':>10} {'Status'}")
for c in customers:
    s = c.summary or {}
    line = (
        f"{s.get('id', ''):>2} "
        f"{s.get('nazwa', ''):<10} "
        f"{str(s.get('czas_zakupow', '')):>8} "
        f"{str(s.get('czas_kolejki', '')):>8} "
        f"{str(s.get('czas_obsługi', '')):>8} "
        f"{str(s.get('czas_w_sklepie', '')):>10} "
        f"{s.get('status', '')}"
    )
    print(line)

## Krok 8 - Symulacja w czasie rzeczywistym

Czasami może zaistnieć potrzeba przeprowadzenia symulacji nie tak szybko, jak to możliwe, ale synchronicznie z czasem na zegarze ściennym. Ten rodzaj symulacji nazywa się również _symulacją w czasie rzeczywistym_.

Aby rozbudować nasze zadanie o _real-time simulation_ w SimPy, wystarczy zastąpić standardowe środowisko symulacyjne `simpy.Environment()` przez `simpy.rt.RealtimeEnvironment()`. Dzięki temu bieżący czas w symulacji będzie odwzorowywał (w wybranej proporcji) rzeczywisty czas zegarowy komputera — symulacja nie będzie wykonywać się „najszybciej jak się da”, ale możliwie synchronicznie z „prawdziwym” czasem.

1. Importujemy `RealtimeEnvironment`
    W naszym kodzie należy dodać:
    
    ```python
    import simpy.rt
    ```
2. Tworzymy środowisko _real-time_
    Zamiast:

    ```python
    env = simpy.Environment()
    ```

    piszemy:

    ```python
    env = simpy.rt.RealtimeEnvironment(factor=1.0, strict=True)
    factor (domyślnie 1.0) – liczba sekund realnego czasu odpowiadająca jednej jednostce czasu symulacji.
    ```

    - Przykład: `factor=0.5` – każda jednostka symulacji trwa 0,5 sekundy zegarowej.
    - `strict` – jeśli `True`, kod generuje błąd, gdy jedna jednostka symulacji trwa w obliczeniach dłużej niż przewiduje to `factor`.

3. Kod główny nie wymaga większych zmian!

4. W efekcie symulacja „wykonuje się” w ustalonym tempie, np. 1 jednostka symulacji = 1 sekunda w realu. Dla prezentacji efektu możesz dodać komunikaty, które lepiej zwizualizują realne upływanie czasu.

:::{tip} Kod zbiorczo:
:class: dropdown

```python
import simpy.rt
import random

# ... (klasa Customer, interrupter itd. jak wcześniej) ...

# Tu zamieniamy środowisko na real-time!
env = simpy.rt.RealtimeEnvironment(factor=1.0, strict=True)

cashiers = simpy.Resource(env, capacity=2)
customers = [
    Customer(env, 1, "Anna", shopping_time=6, checkout_time=2, reaction="leave", checkout_resource=cashiers),
    Customer(env, 2, "Bartek", shopping_time=7, checkout_time=3, reaction="checkout", checkout_resource=cashiers),
    Customer(env, 3, "Celina", shopping_time=5, checkout_time=1, reaction="ignore", checkout_resource=cashiers)
    # ...
]
env.process(interrupter(env, customers[0], interrupt_at=3, reason="Alarm przeciwpożarowy"))
# ...
env.run(until=30)
```
:::

Ostatnia wersja klasy `Customer` nie wypisywała komunikatów. Korzystamy zatem ze zmodyfikowanej jej wersji (zapisującej do logów i wypisującej komunikaty na kosolę)

In [ ]:
import simpy.rt
import random

class Customer(object):
    def __init__(self, env, customer_id, name, shopping_time, checkout_time, reaction, checkout_resource):
        self.env = env
        self.customer_id = customer_id
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.reaction = reaction
        self.checkout_resource = checkout_resource
        self.log = []
        self.summary = {}
        self.action = env.process(self.run())

    def run(self):
        arrival_time = self.env.now
        try:
            msg = f"[{self.customer_id}] {self.name} przychodzi do sklepu"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>5} | {msg}")

            yield self.env.timeout(self.shopping_time)
            shopping_end = self.env.now
            msg = f"[{self.customer_id}] {self.name} kończy zakupy, podchodzi do kasy"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>5} | {msg}")

            # Kolejka do kasy!
            queue_start = self.env.now
            msg = f"[{self.customer_id}] {self.name} oczekuje w kolejce do kasy"
            self.log.append((queue_start, msg))
            print(f"{queue_start:>5} | {msg}")

            with self.checkout_resource.request() as req:
                yield req  # oczekiwanie na kasę

                queue_time = self.env.now - queue_start
                cashier_number = self.checkout_resource.count
                msg = (f"[{self.customer_id}] {self.name} zaczyna obsługę przy kasie {cashier_number} "
                       f"(czekał: {queue_time:.2f})")
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                yield self.env.timeout(self.checkout_time)
                msg = f"[{self.customer_id}] {self.name} opuszcza sklep"
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": shopping_end - arrival_time,
                    "czas_kolejki": queue_time,
                    "czas_obsługi": self.checkout_time,
                    "czas_w_sklepie": self.env.now - arrival_time,
                    "status": "ukończone"
                }
        except simpy.Interrupt as interrupt:
            interrupt_time = self.env.now
            msg = f"[{self.customer_id}] {self.name} PRZERWANIE ({interrupt.cause})"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>5} | {msg}")
            czas_zakupow = interrupt_time - arrival_time
            if self.reaction == "leave":
                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": czas_zakupow,
                    "czas_kolejki": "-",
                    "czas_obsługi": "-",
                    "czas_w_sklepie": czas_zakupow,
                    "status": "przerwane i opuszczone"
                }
            elif self.reaction == "checkout":
                start_kasy = self.env.now
                msg = f"[{self.customer_id}] {self.name} od razu oczekuje w kolejce do kasy (przerwanie)"
                self.log.append((start_kasy, msg))
                print(f"{start_kasy:>5} | {msg}")
                with self.checkout_resource.request() as req:
                    yield req
                    queue_time = self.env.now - start_kasy
                    cashier_number = self.checkout_resource.count
                    msg = (f"[{self.customer_id}] {self.name} (przerwany) zaczyna obsługę przy kasie {cashier_number} "
                           f"(czekał: {queue_time:.2f})")
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    yield self.env.timeout(self.checkout_time)
                    koniec = self.env.now
                    msg = f"[{self.customer_id}] {self.name} opuszcza sklep (po kasie)"
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    self.summary = {
                        "id": self.customer_id,
                        "nazwa": self.name,
                        "czas_zakupow": czas_zakupow,
                        "czas_kolejki": queue_time,
                        "czas_obsługi": self.checkout_time,
                        "czas_w_sklepie": koniec - arrival_time,
                        "status": "przerwane, potem obsługa"
                    }
            elif self.reaction == "ignore":
                remain_shopping = max(0, self.shopping_time - czas_zakupow)
                msg = f"[{self.customer_id}] {self.name} ignoruje przerwanie, kontynuuje zakupy..."
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                yield self.env.timeout(remain_shopping)
                koniec_zakupow = self.env.now
                msg = f"[{self.customer_id}] {self.name} (po przerwaniu) kończy zakupy, podchodzi do kasy"
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                queue_start = self.env.now
                msg = f"[{self.customer_id}] {self.name} (po przerwaniu) oczekuje w kolejce do kasy"
                self.log.append((queue_start, msg))
                print(f"{queue_start:>5} | {msg}")

                with self.checkout_resource.request() as req:
                    yield req
                    queue_time = self.env.now - queue_start
                    cashier_number = self.checkout_resource.count
                    msg = (f"[{self.customer_id}] {self.name} (po przerwaniu) zaczyna obsługę przy kasie {cashier_number} "
                           f"(czekał: {queue_time:.2f})")
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    yield self.env.timeout(self.checkout_time)
                    koniec = self.env.now
                    msg = f"[{self.customer_id}] {self.name} opuszcza sklep (po kasie)"
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    self.summary = {
                        "id": self.customer_id,
                        "nazwa": self.name,
                        "czas_zakupow": self.shopping_time,
                        "czas_kolejki": queue_time,
                        "czas_obsługi": self.checkout_time,
                        "czas_w_sklepie": koniec - arrival_time,
                        "status": "przerwane, ignorowane"
                    }

def interrupter(env, customer, interrupt_at, reason):
    yield env.timeout(interrupt_at)
    customer.action.interrupt(cause=reason)

# Symulacja real-time: 1 jednostka = 1 sek. zegarowego czasu
env = simpy.rt.RealtimeEnvironment(factor=0.5, strict=True)
cashiers = simpy.Resource(env, capacity=2)

# Ustawienia powodujące widoczne kolejki!
customers = [
    Customer(env, 1, "Anna", shopping_time=3, checkout_time=6, reaction="leave", checkout_resource=cashiers),
    Customer(env, 2, "Bartek", shopping_time=3, checkout_time=6, reaction="checkout", checkout_resource=cashiers),
    Customer(env, 3, "Celina", shopping_time=3, checkout_time=6, reaction="ignore", checkout_resource=cashiers),
    Customer(env, 4, "Darek", shopping_time=4, checkout_time=6,
             reaction=random.choice(["leave", "checkout", "ignore"]), checkout_resource=cashiers),
    Customer(env, 5, "Jan", shopping_time=4, checkout_time=6, reaction=None, checkout_resource=cashiers),
]
env.process(interrupter(env, customers[0], interrupt_at=1, reason="Wezwanie na parking"))
env.process(interrupter(env, customers[1], interrupt_at=8, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=5, reason="Awaria techniczna"))


print("="*40)
print("START SYMULACJI: KOLEJKI DO KAS W CZASIE RZECZYWISTYM")
print("="*40)
env.run(until=30)

print("\n" + "="*65)
print("PODSUMOWANIE KLIENTÓW (Czasy w sek. realnego czasu!)")
print("="*65)
print(f"{'ID':>2} {'Nazwa':<10} {'Zakupy':>8} {'Kolejka':>8} {'Obsługa':>8} {'W sklepie':>10} {'Status'}")
for c in customers:
    s = c.summary or {}
    print(
        f"{s.get('id', ''):>2} "
        f"{s.get('nazwa', ''):<10} "
        f"{str(s.get('czas_zakupow', '')):>8} "
        f"{str(s.get('czas_kolejki', '')):>8} "
        f"{str(s.get('czas_obsługi', '')):>8} "
        f"{str(s.get('czas_w_sklepie', '')):>10} "
        f"{s.get('status', '')}"
    )

## Krok 9 - Monitorowanie zasobów

Monitorowanie to stosunkowo złożony temat, mający wiele różnych zastosowań i wiele wariantów. W tym kroku zrealizujemy prostą wersję - monitorowania zasobu kas (np. liczby osób aktualnie obsługiwanych i długości kolejki).

Monitoring oznacza zapisywanie w trakcie symulacji (np. do listy) informacji o tym, co się dzieje z wybranym zasobem lub obiektem – np.:

- ile osób jest obecnie obsługiwanych w kasach,
- ile osób czeka w kolejce,
- w jakim czasie to się dzieje.

To jak obserwowanie i notowanie liczby osób przy kasach i w kolejce w każdym momencie, gdy sytuacja się zmieni.

Jak działa monitoring zasobu (np. kasy)?
Każdy zasób w SimPy (np. Resource czyli kasy) ma pole:

- count — ile stanowisk kasowych jest w tej chwili zajętych,
- queue — lista obiektów czekających w kolejce.

Chcemy za każdym razem, gdy ktoś ustawi się w kolejce lub zwolni kasę, zapisywać:

- aktualny czas symulacji,
- ile kas jest zajętych,
- ile osób czeka w kolejce.

Jak to osiągnąć?
Tworzymy pustą listę do monitorowania:

```python
monitor_data = []
```
Piszemy prostą funkcję, która zapamięta stan kas:

```python
def monitor_resource(data, resource):
    data.append((
        resource._env.now,    # czas symulacji
        resource.count,       # zajęte stanowiska
        len(resource.queue)   # ile osób w kolejce
    ))
```

Wstawiamy “nagrywacza” (`callback`) do zasobu — tzw. _patching_:

Specjalna funkcja podmienia metody zasobu tak, by po każdej zmianie wywoływać nasz monitor:

```python
from functools import partial, wraps

def patch_resource(resource, pre=None, post=None):
    def get_wrapper(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if pre:
                pre(resource)
            ret = func(*args, **kwargs)
            if post:
                post(resource)
            return ret
        return wrapper

    for name in ['request', 'release']:
        if hasattr(resource, name):
            setattr(resource, name, get_wrapper(getattr(resource, name)))

# W kodzie głównym:
monitor = partial(monitor_resource, monitor_data)
patch_resource(cashiers, post=monitor)
```

Efekt:

Za każdym razem, gdy ktoś dołącza do kolejki lub kończy obsługę — do listy monitor_data dopisuje się wpis z czasem, liczbą zajętych kas i liczbą oczekujących.

Dzięki temu możemy na koniec symulacji (lub na bieżąco) zobaczyć:

- Kiedy były największe kolejki,
- Jak zmieniało się obłożenie kas,
- Można tworzyć wykres _ile osób w kolejce_ w czasie.

Przykładowy wynik monitoringu:

```
Czas   Zajęte  Kolejka
    3       2        1
    9       1        0
   15       0        0
```

czyli:
- o czasie 3, dwie kasy zajęte i 1 osoba w kolejce;
- o czasie 9, jedna kasa zajęta, nikogo w kolejce itd.

In [ ]:
Poniżej pełny kod symulacji z monitorowaniem

In [ ]:
import simpy.rt
import random
from functools import partial, wraps

# ---- Monitoring zasobu (patche) ----
def patch_resource(resource, pre=None, post=None):
    def get_wrapper(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if pre:
                pre(resource)
            ret = func(*args, **kwargs)
            if post:
                post(resource)
            return ret
        return wrapper

    # patchujemy metody request, release (można dodać get, put dla innych rodzajów zasobów)
    for name in ['request', 'release']:
        if hasattr(resource, name):
            setattr(resource, name, get_wrapper(getattr(resource, name)))

def monitor_resource(data, resource):
    data.append((
        resource._env.now,   # czas
        resource.count,      # zajęte stanowiska
        len(resource.queue)  # długość kolejki
    ))

# ---- Model klienta ----
class Customer(object):
    def __init__(self, env, customer_id, name, shopping_time, checkout_time, reaction, checkout_resource):
        self.env = env
        self.customer_id = customer_id
        self.name = name
        self.shopping_time = shopping_time
        self.checkout_time = checkout_time
        self.reaction = reaction
        self.checkout_resource = checkout_resource
        self.log = []
        self.summary = {}
        self.action = env.process(self.run())

    def run(self):
        arrival_time = self.env.now
        try:
            msg = f"[{self.customer_id}] {self.name} przychodzi do sklepu"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>5} | {msg}")

            yield self.env.timeout(self.shopping_time)
            shopping_end = self.env.now
            msg = f"[{self.customer_id}] {self.name} kończy zakupy, podchodzi do kasy"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>5} | {msg}")

            queue_start = self.env.now
            msg = f"[{self.customer_id}] {self.name} oczekuje w kolejce do kasy"
            self.log.append((queue_start, msg))
            print(f"{queue_start:>5} | {msg}")

            with self.checkout_resource.request() as req:
                yield req  # oczekiwanie na kasę
                queue_time = self.env.now - queue_start
                cashier_number = self.checkout_resource.count
                msg = (f"[{self.customer_id}] {self.name} zaczyna obsługę przy kasie {cashier_number} "
                       f"(czekał: {queue_time:.2f})")
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                yield self.env.timeout(self.checkout_time)
                msg = f"[{self.customer_id}] {self.name} opuszcza sklep"
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": shopping_end - arrival_time,
                    "czas_kolejki": queue_time,
                    "czas_obsługi": self.checkout_time,
                    "czas_w_sklepie": self.env.now - arrival_time,
                    "status": "ukończone"
                }
        except simpy.Interrupt as interrupt:
            interrupt_time = self.env.now
            msg = f"[{self.customer_id}] {self.name} PRZERWANIE ({interrupt.cause})"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>5} | {msg}")
            czas_zakupow = interrupt_time - arrival_time
            if self.reaction == "leave":
                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": czas_zakupow,
                    "czas_kolejki": "-",
                    "czas_obsługi": "-",
                    "czas_w_sklepie": czas_zakupow,
                    "status": "przerwane i opuszczone"
                }
            elif self.reaction == "checkout":
                start_kasy = self.env.now
                msg = f"[{self.customer_id}] {self.name} od razu oczekuje w kolejce do kasy (przerwanie)"
                self.log.append((start_kasy, msg))
                print(f"{start_kasy:>5} | {msg}")

                with self.checkout_resource.request() as req:
                    yield req
                    queue_time = self.env.now - start_kasy
                    cashier_number = self.checkout_resource.count
                    msg = (f"[{self.customer_id}] {self.name} (przerwany) zaczyna obsługę przy kasie {cashier_number} "
                           f"(czekał: {queue_time:.2f})")
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    yield self.env.timeout(self.checkout_time)
                    koniec = self.env.now
                    msg = f"[{self.customer_id}] {self.name} opuszcza sklep (po kasie)"
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    self.summary = {
                        "id": self.customer_id,
                        "nazwa": self.name,
                        "czas_zakupow": czas_zakupow,
                        "czas_kolejki": queue_time,
                        "czas_obsługi": self.checkout_time,
                        "czas_w_sklepie": koniec - arrival_time,
                        "status": "przerwane, potem obsługa"
                    }
            elif self.reaction == "ignore":
                remain_shopping = max(0, self.shopping_time - czas_zakupow)
                msg = f"[{self.customer_id}] {self.name} ignoruje przerwanie, kontynuuje zakupy..."
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                yield self.env.timeout(remain_shopping)
                koniec_zakupow = self.env.now
                msg = f"[{self.customer_id}] {self.name} (po przerwaniu) kończy zakupy, podchodzi do kasy"
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>5} | {msg}")

                queue_start = self.env.now
                msg = f"[{self.customer_id}] {self.name} (po przerwaniu) oczekuje w kolejce do kasy"
                self.log.append((queue_start, msg))
                print(f"{queue_start:>5} | {msg}")

                with self.checkout_resource.request() as req:
                    yield req
                    queue_time = self.env.now - queue_start
                    cashier_number = self.checkout_resource.count
                    msg = (f"[{self.customer_id}] {self.name} (po przerwaniu) zaczyna obsługę przy kasie {cashier_number} "
                           f"(czekał: {queue_time:.2f})")
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    yield self.env.timeout(self.checkout_time)
                    koniec = self.env.now
                    msg = f"[{self.customer_id}] {self.name} opuszcza sklep (po kasie)"
                    self.log.append((self.env.now, msg))
                    print(f"{self.env.now:>5} | {msg}")

                    self.summary = {
                        "id": self.customer_id,
                        "nazwa": self.name,
                        "czas_zakupow": self.shopping_time,
                        "czas_kolejki": queue_time,
                        "czas_obsługi": self.checkout_time,
                        "czas_w_sklepie": koniec - arrival_time,
                        "status": "przerwane, ignorowane"
                    }

def interrupter(env, customer, interrupt_at, reason):
    yield env.timeout(interrupt_at)
    customer.action.interrupt(cause=reason)

# --- Środowisko, zasoby, MONITORING ---
env = simpy.rt.RealtimeEnvironment(factor=0.5, strict=True)
cashiers = simpy.Resource(env, capacity=2)
monitor_data = []
monitor = partial(monitor_resource, monitor_data)
patch_resource(cashiers, post=monitor)

# Testowe czasy wymuszające kolejki
customers = [
    Customer(env, 1, "Anna", shopping_time=3, checkout_time=6, reaction="leave", checkout_resource=cashiers),
    Customer(env, 2, "Bartek", shopping_time=3, checkout_time=6, reaction="checkout", checkout_resource=cashiers),
    Customer(env, 3, "Celina", shopping_time=3, checkout_time=6, reaction="ignore", checkout_resource=cashiers),
    Customer(env, 4, "Darek", shopping_time=4, checkout_time=6,
             reaction=random.choice(["leave", "checkout", "ignore"]), checkout_resource=cashiers),
]

env.process(interrupter(env, customers[1], interrupt_at=8, reason="Nagła potrzeba"))
env.process(interrupter(env, customers[2], interrupt_at=5, reason="Awaria techniczna"))
env.process(interrupter(env, customers[3], interrupt_at=9, reason="Wezwanie na parking"))

print("="*42)
print("START SYMULACJI: KOLEJKI DO KAS i MONITORING")
print("="*42)
env.run(until=30)

print("\n" + "="*66)
print("PODSUMOWANIE KLIENTÓW (czasy w sek. realnego czasu!)")
print("="*66)
print(f"{'ID':>2} {'Nazwa':<10} {'Zakupy':>8} {'Kolejka':>8} "
      f"{'Obsługa':>8} {'W sklepie':>10} {'Status'}")
for c in customers:
    s = c.summary or {}
    print(
        f"{s.get('id', ''):>2} "
        f"{s.get('nazwa', ''):<10} "
        f"{str(s.get('czas_zakupow', '')):>8} "
        f"{str(s.get('czas_kolejki', '')):>8} "
        f"{str(s.get('czas_obsługi', '')):>8} "
        f"{str(s.get('czas_w_sklepie', '')):>10} "
        f"{s.get('status', '')}"
    )

print("\nMONITORING KAS (czas, zajęte stanowiska, długość kolejki):")
print(f"{'Czas':>5} {'Zajęte':>7} {'Kolejka':>8}")
for t, busy, qlen in monitor_data:
    print(f"{t:>5} {busy:>7} {qlen:>8}")


## Krok 10. Czasy zadane jako rozkłady prawdopodobieństw

Aby zamiast czasów deterministycznych (np. zawsze 3 sekundy na zakupy) podpiąć rozkłady prawdopodobieństw (czyli żeby długości zakupów i obsługi były losowane z rozkładów statystycznych), wystarczy:

- nie przekazywać do klienta stałych wartości, tylko funkcje generujące czas z rozkładu
(np. `random.expovariate`, `random.gauss`, `random.randint` itd.),
- wywoływać generator rozkładu w miejscu, gdzie czas jest używany
(czyli w metodzie `run` klienta).

Wskazówki:

- Podmieniamy wywołania na `yield env.timeout(funkcja_rozkladu())` zamiast na stałą liczbę.
- dla większej kontroli można przekazywać typ rozkładu i parametry (np. argumenty do generatorów).

Podstawowe rozkłady w module random:
- random.expovariate(lambd) – wykładniczy,
- random.gauss(mu, sigma) – normalny,
- random.uniform(a, b) – jednostajny,
- random.randint(a, b) – skończony dyskretny.



In [ ]:
# rozkład wykładniczy i normalny

import simpy
import random
from functools import partial, wraps

# ---- Monitoring zasobu (patche) ----
def patch_resource(resource, pre=None, post=None):
    def get_wrapper(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            if pre:
                pre(resource)
            ret = func(*args, **kwargs)
            if post:
                post(resource)
            return ret
        return wrapper

    for name in ['request', 'release']:
        if hasattr(resource, name):
            setattr(resource, name, get_wrapper(getattr(resource, name)))

def monitor_resource(data, resource):
    data.append((
        resource._env.now,
        resource.count,
        len(resource.queue)
    ))

# ---- Rozkłady dla czasów zakupów/obsługi ----
def zakupy_exponential():
    return random.expovariate(1/5)  # średni czas zakupów 5 jednostek

def kasa_normal():
    return max(1, random.gauss(4, 1))  # czas obsługi: średnia 4, min. 1

# ---- Model klienta ----
class Customer(object):
    def __init__(self, env, customer_id, name, shopping_time_fun, checkout_time_fun, reaction, checkout_resource):
        self.env = env
        self.customer_id = customer_id
        self.name = name
        self.shopping_time_fun = shopping_time_fun
        self.checkout_time_fun = checkout_time_fun
        self.reaction = reaction
        self.checkout_resource = checkout_resource
        self.log = []
        self.summary = {}
        self.action = env.process(self.run())

    def run(self):
        arrival_time = self.env.now
        try:
            msg = f"[{self.customer_id}] {self.name} przychodzi do sklepu"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>6.2f} | {msg}")

            shopping_time = self.shopping_time_fun()
            yield self.env.timeout(shopping_time)
            shopping_end = self.env.now
            msg = f"[{self.customer_id}] {self.name} kończy zakupy po {shopping_time:.2f}, podchodzi do kasy"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>6.2f} | {msg}")

            queue_start = self.env.now
            msg = f"[{self.customer_id}] {self.name} oczekuje w kolejce do kasy"
            self.log.append((queue_start, msg))
            print(f"{queue_start:>6.2f} | {msg}")

            with self.checkout_resource.request() as req:
                yield req  # oczekiwanie na kasę
                queue_time = self.env.now - queue_start
                cashier_number = self.checkout_resource.count
                checkout_time = self.checkout_time_fun()
                msg = (f"[{self.customer_id}] {self.name} zaczyna obsługę przy kasie {cashier_number} "
                       f"(czekał: {queue_time:.2f}), czas obsługi {checkout_time:.2f}")
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>6.2f} | {msg}")

                yield self.env.timeout(checkout_time)
                msg = f"[{self.customer_id}] {self.name} opuszcza sklep"
                self.log.append((self.env.now, msg))
                print(f"{self.env.now:>6.2f} | {msg}")

                self.summary = {
                    "id": self.customer_id,
                    "nazwa": self.name,
                    "czas_zakupow": shopping_end - arrival_time,
                    "czas_kolejki": queue_time,
                    "czas_obsługi": checkout_time,
                    "czas_w_sklepie": self.env.now - arrival_time,
                    "status": "ukończone"
                }
        except simpy.Interrupt as interrupt:
            interrupt_time = self.env.now
            msg = f"[{self.customer_id}] {self.name} PRZERWANIE ({interrupt.cause})"
            self.log.append((self.env.now, msg))
            print(f"{self.env.now:>6.2f} | {msg}")
            czas_zakupow = interrupt_time - arrival_time
            self.summary = {
                "id": self.customer_id,
                "nazwa": self.name,
                "czas_zakupow": czas_zakupow,
                "czas_kolejki": "-",
                "czas_obsługi": "-",
                "czas_w_sklepie": czas_zakupow,
                "status": "przerwane"
            }

def interrupter(env, customer, interrupt_at, reason):
    yield env.timeout(interrupt_at)
    customer.action.interrupt(cause=reason)

# --- Środowisko, zasoby, MONITORING ---
env = simpy.Environment()
cashiers = simpy.Resource(env, capacity=2)
monitor_data = []
monitor = partial(monitor_resource, monitor_data)
patch_resource(cashiers, post=monitor)

# Wariant: 6 klientów, czasy losowane!
customers = [
    Customer(env, 1, "Anna", zakupy_exponential, kasa_normal, "leave", cashiers),
    Customer(env, 2, "Bartek", zakupy_exponential, kasa_normal, "checkout", cashiers),
    Customer(env, 3, "Celina", zakupy_exponential, kasa_normal, "ignore", cashiers),
    Customer(env, 4, "Darek", zakupy_exponential, kasa_normal, random.choice(["leave", "checkout", "ignore"]), cashiers),
    Customer(env, 5, "Ela", zakupy_exponential, kasa_normal, "ignore", cashiers),
    Customer(env, 6, "Franek", zakupy_exponential, kasa_normal, "leave", cashiers),
]
# (opcjonalne przerwania można odkomentować)
# env.process(interrupter(env, customers[0], interrupt_at=7, reason="Alarm przeciwpożarowy"))

print("="*42)
print("START SYMULACJI (rozkłady losowe): KOLEJKI I MONITORING")
print("="*42)
env.run(until=60)

print("\nLOGI ZDARZEŃ:")
print("-"*40)
for c in customers:
    print(f"\n[Klient {c.customer_id}] {c.name}:")
    for t, desc in c.log:
        print(f"{t:>6.2f} | {desc}")

print("\n" + "="*66)
print("PODSUMOWANIE KLIENTÓW (czasy w jednostkach symulacji)")
print("="*66)
print(f"{'ID':>2} {'Nazwa':<10} {'Zakupy':>8} {'Kolejka':>8} "
      f"{'Obsługa':>8} {'W sklepie':>10} {'Status'}")
for c in customers:
    s = c.summary or {}
    print(
        f"{s.get('id', ''):>2} "
        f"{s.get('nazwa', ''):<10} "
        f"{str(round(s.get('czas_zakupow', 0),2) if 'czas_zakupow' in s and s.get('czas_zakupow', '-')!='-' else '-'):>8} "
        f"{str(round(s.get('czas_kolejki', 0),2) if 'czas_kolejki' in s and s.get('czas_kolejki', '-')!='-' else '-'):>8} "
        f"{str(round(s.get('czas_obsługi', 0),2) if 'czas_obsługi' in s and s.get('czas_obsługi', '-')!='-' else '-'):>8} "
        f"{str(round(s.get('czas_w_sklepie', 0),2) if 'czas_w_sklepie' in s and s.get('czas_w_sklepie', '-')!='-' else '-'):>10} "
        f"{s.get('status', '')}"
    )

print("\nMONITORING KAS (czas, zajęte stanowiska, długość kolejki):")
print(f"{'Czas':>6} {'Zajęte':>7} {'Kolejka':>8}")
for t, busy, qlen in monitor_data:
    print(f"{t:>6.2f} {busy:>7} {qlen:>8}")


## Zadanie: Symulacja działania wind w budynku biurowym

Wzorując się na przykładzie, opracuj model symulacyjny dla nastepującego problemu:


Stwórz model symulacji, w którym pracownicy budynku biurowego **korzystają z wind**, by przemieszczać się między piętrami. Twoja symulacja powinna obejmować:

- **Procesy**: Pracownik przywołuje windę, czeka na transport, przemieszcza się na wybrane piętro.
- **Shared resources**: Kilka wind obsługuje wszystkich pracowników (liczba wind do wyboru).
- **Losowe czasy**: Czas pojawienia się pracownika na dolnym holu, czas jazdy windy (z rozkładów losowych), czas oczekiwania w kolejce.
- **Kolejka**: Jeśli nie ma wolnej windy, pracownicy oczekują.
- **Monitoring**: Zbieraj przebieg wydarzeń – ile wind jest zajętych w danym momencie, ile osób czeka na windę, czasy oczekiwania.
- **Podsumowanie**: Po zakończeniu symulacji wypisz statystyki – średni czas oczekiwania na windę, maksymalną kolejkę, liczbę obsłużonych osób, czas pobytu w windzie.

**Wskazówki do implementacji**

1. **Model wind jako zasobów SimPy (`Resource`)**. Przykładowo, możesz założyć 2 lub 3 windy na budynek.
2. **Proces pracownika**:
    - Pracownik pojawia się na parterze w losowym momencie (np. rozkład wykładniczy).
    - Pracownik zgłasza żądanie windy (w SimPy: `with wind.request() as req:`).
    - Jeśli jest dostępna, jedzie. Jeśli nie, staje w kolejce; zanotuj czas oczekiwania.
    - Czas jazdy windy wybierz losowo (np. rozkład normalny).
    - Po wyjściu z windy rejestruj czas obsługi.
3. **Monitorowanie**:
    - Twórz logi z przebiegu – kto kiedy czekał, startował, opuszczał windę.
    - Dokonuj monitoring zasobu: ile wind jest zajętych, ile osób czeka na windę.
4. **Losowe czasy i parametry:**
    - Rozkład czasów pojawiania się osób na parterze: wykładniczy.
    - Rozkład jazdy windy: normalny (lub jednostajny w zakresie dla uproszczenia).
5. **Statystyki po symulacji**:
    - Uśredniony czas oczekiwania na windę.
    - Maksymalna zanotowana długość kolejki.
    - Liczba obsłużonych osób.
    - Średni czas pobytu w windzie.

**Wyzwania dodatkowe**

- Dodaj “VIP” pracownika z priorytetem do wind.
- Dodaj awarię jednej windy w trakcie symulacji.
- Zróżnicuj liczbę pięter i losuj docelowe piętro pracownika.
- Dodaj opcję rezygnacji z czekania jeśli czas przekroczy X jednostek.
- Zbierz monitoring do pandas i stwórz wykres: długość kolejki do windy w czasie.
